# Implementación de API REST

Implementación de una API REST utilizando Flask.
    Documentacion de apis
Presentación y estructuración del trabajo final.

Modules para ver ese dia: flask-restful, flask-restful-swagger
Material: https://blog.miguelgrinberg.com/post/designing-a-restful-api-with-python-and-flask

# ¿Qué es REST?

> La **Transferencia de Estado Representacional** (Representational State Transfer) o **REST** es un estilo de arquitectura software para sistemas hipermedia distribuidos como la World Wide Web.

Existen, otros tipos de métodos para implementar servicios web, como **RPC**, **SOAP** o **WSDL**. Sin embargo, el uso de dichos mecanismos no se suele recomendar en favor de RESTful, ya que RESTful es mucho más fácil de entender e implementar.

Características del diseño de la arquitectura REST:
1. **Cliente/Servidor** separación clara entre los dos agentes básicos en el intercambio de información.
1. **Stateless** el servidor no tiene porqué almacenar datos del cliente para mantener un estado del mismo.
1. **Cacheable** el servidor debe poder definir algún modo de cachear dichas peticiones, para aumentar el rendimiento, escalabilidad, etc.
1. **Sistema por capas** el cliente no necesita saber cuales son las capas implicadas en la obtención de la información.
1. **Interfaz uniforme** la interfaz de comunicación entre un cliente y el servidor no debe depender ni del servidor ni del cliente, se debe cumplir una interfaz definida de antemano

Estos principios de diseño ya vienen dados por el uso del protocolo **HTTP** ya que:

* **Identificación de recursos** una aplicación REST debe poder identificar sus recursos de manera uniforme. HTTP implementa esto usando las llamadas URIs o las tradicionales [URLs](https://es.wikipedia.org/wiki/Localizador_de_recursos_uniforme).
* **Recursos y representaciones:** REST define también la manera en que podemos interactuar con la representación de un recurso, ya sea para crearlo, obtenerlo, editarlo o borrarlo. HTTP define distintos verbos y un contenido en la respuesta lo que permite modelar las operaciones con el recurso en el caso de que tengamos permiso para hacerlo.
* **Mensajes autodescriptivos** cuando se hacen peticiones a un servidor, éste debería devolver una respuesta que permita entender sin lugar a duda cual ha sido el resultado de la operación, así como si dicha operación es cacheable, si ha habido algún error, etc. HTTP implementa esto a través del estado y una serie de cabeceras. El uso de las cabeceras depende de la implementación, REST no fuerza el contenido de las cabeceras, aunque existen algunas convenciones [básicas](http://www.restapitutorial.com/httpstatuscodes.html).
* **HATEOAS** es la necesidad de incluir en las respuestas del servidor toda aquella información que necesita el cliente para seguir operando con el servicio web. La mayoría de servicios web no cumplen con esta necesidad. 

```json
{
    "id": 4,
    "nombre": "Juan",
    "apellido": "Carlos",
    "coches": [
    	{
    		"id": 1253
    	}
    ]
}
```

```json
{
    "id": 78,
    "nombre": "Juan",
    "apellido": "García",
    "coches": [
    	{
    		"coche": "http://miservidor/concesionario/api/v1/clientes/78/coches/1033"
    	}
    ]
}
```

# ¿Qué es un recurso?


# ¿Qué es un servicio web RESTful?

* **GET** - Obtener información sobre un recurso - http://api.com/api/orders (recuperar la lista de pedidos)
* **GET** - Obtener información sobre un recurso - http://api.com/api/orders/123 (recuperar orden # 123)
* **POST** - Crear un nuevo recurso	- http://api.com/api/orders (crear un nuevo pedido, a partir de los datos proporcionados con la solicitud)
* **PUT** - Actualizar un recurso - http://api.com/api/orders/123 (actualización de la orden # 123, a partir de los datos proporcionados con la solicitud)
* **DELETE** - Eliminar un recurso - http://api.com/api/orders/123 (borrar pedido # 123)

# Servicio web sencillo

* [A RESTful Tutorial](http://www.restapitutorial.com/)